# Lab 7: Object Detection

Object detection is a fundamental task in computer vision that involves identifying and locating objects of interest within images or video frames. It plays a crucial role in various real-world applications, such as autonomous vehicles, video surveillance, image-based search, robotics, medical image analysis, and augmented reality. By accurately detecting and localizing objects, computer vision systems can better understand the visual world and make informed decisions based on the detected objects' positions, sizes, and relationships.

# Dataset

We will use the Vehicle-OpenImages dataset from Roboflow. The dataset contains images of various vehicles in varied traffic conditions. These images have been collected from the Open Image dataset. The images are from varied conditions and scenes. It contains 5 classes in total. They are Car, Bus, Motorcycle, Truck, Ambulance.


### Download and unzip the Dataset

For each of the train, validation, and test set folders within the dataset, you need to create two subfolders:

1. "images": This folder should contain all the images corresponding to the specific set (train, validation, or test).
2. "labels": This folder should have a ".txt" file for every image in the "images" folder. Each label file will include one line per bounding box, with the bounding box information in the YOLO format as follows:

object_class  x  y  width  height

* "object_class": This is an integer representing the object's class. Start with 0 for the first class and increment by 1 for each additional unique class in the dataset.
* "x" and "y": These values represent the normalized coordinates of the bounding box's center, with respect to the image width and height, respectively. Ensure that the coordinates are within the range of [0, 1].
* "width" and "height": These values represent the normalized width and height of the bounding box, with respect to the image width and height, respectively. Ensure that these dimensions are within the range of [0, 1].

In [ ]:
!gdown https://drive.google.com/file/d/16BZkCbPQNUwTEOYXsA2cPavUPQys0Acr/view?usp=sharing --fuzzy

In [ ]:
!unzip /content/Vehicles-OpenImages.zip -d ./vehicles

# Annotation
In order to annotate your own dataset use the labelimg tool

# YOLOv5

Clone the YOLOv5 Repository

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Install dependency from requirements.txt file

In [ ]:
!pip install -r yolov5/requirements.txt

## Training Options
* img        : Size of image. The image is a square one. The original image is resized while maintaining the aspect ratio. The longer side of the image is resized to this number. The shorter side is padded with grey color.

* batch: The batch size

* epochs: Number of epochs to train for

* data: Data YAML file that contains information about the
dataset (path of images, labels)

* workers: Number of CPU workers

* cfg: Model architecture. There are 4 choices available: yolo5s.yaml, yolov5m.yaml, yolov5l.yaml, yolov5x.yaml. The size and complexity of these models increases in the ascending order and you can choose a model which suits the complexity of your object detection task. In case you want to work with a custom architecture, you will have to define a YAML file in the models folder specifying the network architecture.

* weights: Pretrained weights you want to start training from. If you want to train from scratch, use --weights ' '

* name: Various things about training such as train logs. Training weights would be stored in a folder named runs/train/name

* hyp: YAML file that describes hyperparameter choices. For examples of how to define hyperparameters, see data/hyp.scratch.yaml. If unspecified, the file data/hyp.scratch.yaml is used.

## Data Config File

* Details for the dataset you want to train your model on are defined by the data config YAML file. The following parameters have to be defined in a data config file:

* train, test, and val: Locations of train, test, and validation images.

* nc: Number of classes in the dataset.

* names: Names of the classes in the dataset. The index of the
classes in this list would be used as an identifier for the class names in the code.

## Hyperparameter Config File
The hyperparameter config file helps us define the hyperparameters for our neural network. We are going to use the default one, data/hyp.scratch.yaml. This is what it looks like.

## Custom Network Architecture
YOLO v5 also allows you to define your own custom architecture and anchors if one of the pre-defined networks doesn't fit the bill for you. For this you will have to define a custom weights config file. For this example, we use the the yolov5s.yaml. This is what it looks like.

## Train Model from pretrained model
The models range from nano to large as follows:


*   yolov5n
*   yolov5s
* yolov5m
* yolov5l



### Important Note
Please provide absolute paths for train and validation datasets in "data.yaml" file of you dataset.

To train the model we use the "train.py" file and provide various arguments listed above.

In [ ]:
!python ./yolov5/train.py --img 640  --batch 32 --epochs 10 --data ./vehicles/data.yaml --weights 'yolov5n.pt' --name vehicles

## Perform validation on test/val set

To validate your model on the test set change the path of the "val" in your "data.yaml" file to the test set directory. If you want to validate it on validation set again, keep the validation path against the "val" variable.

In [ ]:
!python ./yolov5/val.py --img 640  --batch 32 --data ./vehicles/data.yaml --weights '/content/yolov5/runs/train/vehicles2/weights/best.pt' --name vehicles

## Perform inference using trained model

To perform inference using our models, we use the "detect.py" file. By providing the source path to a folder, video or a single image, it will perform detection on all the images in the folder, or the complete video or the single image.

In [ ]:
!python ./yolov5/detect.py --weights '/content/yolov5/runs/train/vehicles2/weights/best.pt' --source /content/vehicles/test/images

# YOLOv8

YOLOv8 is available in the form of a python package for ease of use. You can use tensorboard with each of the YOLO models given in this repo. You just have to provide the path of the specific directory containing the training experiments. In the cell below, before starting training we create the experiment directories ourselves so that we can start tensorboard before starting the model training for tracking model learning in real time

In [ ]:
!mkdir runs2
!mkdir runs2/detect

In [ ]:
import tensorboard
%load_ext tensorboard
%tensorboard --logdir /content/runs/detect
# Don't worry if tensor board is not working for your. You may ignore this error and move on to the next cells.

In [ ]:
# This is the ultralytics library for using Ultralytics' various tools for object detection.
!pip install ultralytics

## Train YOLOv8 Model
The YOLOv8 can be trained, validated and used for inferenced by either using the CLI or its python API.

1. Start model training of nano model from scratch using command line:

In [ ]:
!yolo task=detect mode=train model=yolov8n.yaml imgsz=640 data=/content/vehicles/data.yaml epochs=10 batch=8 name=yolov8n_scratch

2. Start training by using the "ultralytics" package in python.

In [ ]:
from ultralytics import YOLO
# Load the model.
model = YOLO('yolov8n.pt')

# Training.
results = model.train(
   data='/content/vehicles/data.yaml',
   imgsz=640,
   epochs=10,
   batch=64,
   name='vehicles_pretrained')

## Validate YOLOv8 Model
Validate the model on val/test. You need to change the path for the "val" variable to the test set in the "data.yaml" file

In [ ]:
# Load the model.
model = YOLO('/content/yolov8n.pt')

# Training.
results = model.val(
   data='/content/vehicles/data.yaml',
   imgsz=640,
   epochs=10,
   batch=64,
   name='vehicles_pretrained')

## Perform Inference Using YOLOv8
We use CLI for inferencing on the images in the test set

In [ ]:
!yolo predict model=/content/yolov8n.pt source='/content/vehicles/test/images'

You can also use the python code for this but it will return the predictions as a list.

In [ ]:
model = YOLO('/content/yolov8n.pt')
results = model.predict(
   source='/content/vehicles/test/images')


# Tasks:


1.   Please fully train the YOLOv5s (small) and YOLOv8s (small) models on the provided dataset
2.   Evaluate your models on the test sets and provide the mAP values.
3.   Prepare a report in which you compare the performance of both the models. Please study the architectures of both YOLOV5 and YOLOV8 and illustrate these in your report.
4. In your report, discuss which model is performing better on the test set and what can be the reason? If YOLOV8 is performing better, then what are the innovations that allow YOLOV8 to perform better than YOLOV5.
5. Show predictions of both models on 4-5 test images in the report and compare them side by side.  



***This is an individual lab.***

# Source

https://github.com/Vision-At-SEECS/Pytorch_Labs/blob/main/Lab5_CV_Object_Detection.ipynb

Prepared by: Bostan Khan, Team Lead, Machine Vision and Intelligent Systems Lab